In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format


config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

In [6]:
input_data="s3a://udacity-dend/"
song_data = os.path.join(input_data,"song_data/A/A/A/*")

In [30]:
import zipfile
with zipfile.ZipFile(log_data,"r") as zip_ref:
    zip_ref.extractall("data")

In [7]:
df = spark.read.json(song_data)

In [8]:
df.printSchema()
df.show(5)

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|ARTC1LV1187B9A4858|        51.4536|Goldsmith's Colle...|        -0.01802|  The Bonzo Dog Band|3

In [11]:
songs_table = df.select("song_id","title","artist_id","year","duration")

In [12]:
songs_table.head()

Row(song_id='SOAFBCP12A8C13CC7D', title='King Of Scurf (2007 Digital Remaster)', artist_id='ARTC1LV1187B9A4858', year=1972, duration=301.40036)

In [13]:
songs_table.show(5)

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOAFBCP12A8C13CC7D|King Of Scurf (20...|ARTC1LV1187B9A4858|1972|301.40036|
|SOKTJDS12AF72A25E5|Drown In My Own T...|ARA23XO1187B9AF18F|   0|  192.522|
|SOEKAZG12AB018837E|I'll Slap Your Fa...|ARSVTNL1187B992A91|2001|129.85424|
|SOQPWCR12A6D4FB2A3|A Poor Recipe For...|AR73AIO1187B9AD57B|2005|118.07302|
|SOBRKGM12A8C139EF6|Welcome to the Pl...|ARXQBR11187B98A2CC|1985|821.05424|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows



In [17]:
output_data = "s3a://aws-emr-resources-714348276754-us-east-1/"
songs_table.write.partitionBy("year","artist_id").parquet(os.path.join(output_data,"songs.parquet"),'overwrite')

In [21]:
    log_data = os.path.join(input_data,"log_data/*/*/*")

    # read log data file
    df = spark.read.json(log_data)

In [58]:
artists_table =df.select("artist_id","artist_name","artist_location","artist_latitude","artist_longitude")

In [59]:
artists_table.write.format("parquet").save("data/artists.parquet")

In [30]:
log_data = "data/log_data"

In [3]:
df = spark.read.json('data/log_data/*')

In [13]:
df.count()

8056

In [4]:
df=df.where(df.page=="NextSong")

In [16]:
df.count()

6820

In [17]:
users_table =df.select("userid","firstname","lastname","gender","level")

In [18]:
users_table.write.format("parquet").save("data/users.parquet")

In [19]:
from pyspark.sql.functions import to_timestamp
time=df.select( to_timestamp((df['ts']/1000).cast('timestamp'), "yyyy-MM-dd hh:mm:ss").alias('start_time'))
df=df.withColumn("start_time",time['start_time'])

AnalysisException: 'Resolved attribute(s) start_time#279 missing from itemInSession#10L,lastName#11,auth#7,sessionId#18L,firstName#8,userId#23,location#14,registration#17,gender#9,status#20L,level#13,artist#6,ts#21L,userAgent#22,page#16,length#12,song#19,method#15 in operator !Project [artist#6, auth#7, firstName#8, gender#9, itemInSession#10L, lastName#11, length#12, level#13, location#14, method#15, page#16, registration#17, sessionId#18L, song#19, status#20L, ts#21L, userAgent#22, userId#23, start_time#279 AS start_time#281].;;\n!Project [artist#6, auth#7, firstName#8, gender#9, itemInSession#10L, lastName#11, length#12, level#13, location#14, method#15, page#16, registration#17, sessionId#18L, song#19, status#20L, ts#21L, userAgent#22, userId#23, start_time#279 AS start_time#281]\n+- Filter (page#16 = NextSong)\n   +- Relation[artist#6,auth#7,firstName#8,gender#9,itemInSession#10L,lastName#11,length#12,level#13,location#14,method#15,page#16,registration#17,sessionId#18L,song#19,sta

In [12]:
from pyspark.sql.functions import from_unixtime
df.select(from_unixtime(df['ts']/1000,format='yyyy-MM-dd HH:mm:ss').alias('start_time'))

+-------------------+
|         start_time|
+-------------------+
|2018-11-15 00:30:26|
|2018-11-15 00:41:21|
|2018-11-15 00:45:41|
|2018-11-15 03:44:09|
|2018-11-15 05:48:55|
+-------------------+
only showing top 5 rows



In [12]:
date=time.select(time['start_time'], hour(time['start_time']).alias('hour'), dayofmonth(time['start_time']).alias('day'), weekofyear(time['start_time']).alias('week'), month(time['start_time']).alias('month'),year(time['start_time']).alias('year') )
date.show(5)

+-------------------+----+---+----+-----+----+
|         start_time|hour|day|week|month|year|
+-------------------+----+---+----+-----+----+
|2018-11-15 00:30:26|   0| 15|  46|   11|2018|
|2018-11-15 00:41:21|   0| 15|  46|   11|2018|
|2018-11-15 00:45:41|   0| 15|  46|   11|2018|
|2018-11-15 03:44:09|   3| 15|  46|   11|2018|
|2018-11-15 05:48:55|   5| 15|  46|   11|2018|
+-------------------+----+---+----+-----+----+
only showing top 5 rows



In [24]:
get_timestamp = udf (lambda x: datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M:%S'))
new_df=df.withColumn('timest',get_timestamp(df['ts']))

In [13]:
date.write.format("parquet").save("data/time.parquet")

In [15]:
song_df=spark.read.parquet("data/songs.parquet")

In [25]:
new_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timest: string (nullable = true)



In [26]:
song_df.printSchema()

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)



In [27]:
song_df.show(5)

+------------------+--------------------+---------+----+------------------+
|           song_id|               title| duration|year|         artist_id|
+------------------+--------------------+---------+----+------------------+
|SOAOIBZ12AB01815BE|I Hold Your Hand ...| 43.36281|2000|ARPBNLO1187FB3D52F|
|SONYPOM12A8C13B2D7|I Think My Wife I...|186.48771|2005|ARDNS031187B9924F0|
|SODREIN12A58A7F2E5|A Whiter Shade Of...|326.00771|   0|ARLTWXK1187FB5A3F8|
|SOYMRWW12A6D4FAB14|The Moon And I (O...| 267.7024|   0|ARKFYS91187B98E58F|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|279.97995|   0|ARPFHN61187FB575F6|
+------------------+--------------------+---------+----+------------------+
only showing top 5 rows



In [28]:
new_df.show(5)

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|             timest|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-15 00:30:26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smi

In [34]:
from pyspark.sql.functions import monotonically_increasing_id
new_df.createOrReplaceTempView("new_df_table")
song_df.createOrReplaceTempView("song_df_table")
songplays_table =spark.sql('''select monotonically_increasing_id() as songplay_id, timest as start_time
, userId as user_id,level, song_id, artist_id, sessionid as session_id, location,userAgent as user_agent
from new_df_table a join song_df_table b on lower(a.song)=lower(b.title)  ''')

In [35]:
songplays_table.show(5)

+-----------+-------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|songplay_id|         start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|
+-----------+-------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|          0|2018-11-21 21:56:47|     15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|
|          1|2018-11-14 05:06:03|     10| free|SOGDBUF12A8C140FAA|AR558FS1187FB45658|       484|Washington-Arling...|"Mozilla/5.0 (Mac...|
|          2|2018-11-19 09:14:20|     24| paid|SOGDBUF12A8C140FAA|AR558FS1187FB45658|       672|Lake Havasu City-...|"Mozilla/5.0 (Win...|
|          3|2018-11-27 22:35:59|     80| paid|SOGDBUF12A8C140FAA|AR558FS1187FB45658|       992|Portland-South Po...|"Mozilla/5.0 (Mac...|
+-----------+--------------